In [28]:
import cv2
import os 
import time
import random
import numpy as np

from skimage import color
from skimage.feature import local_binary_pattern
from scipy.spatial import distance
from matplotlib import pyplot as plt
from IPython.display import clear_output
from pathlib import Path

## Functions for creating random patches from images 

Most of these functions are reused from the github page: https://github.com/i-evi/p2gan, based on the p2GAN paper: https://arxiv.org/pdf/2001.07466.pdf. With some small adaptdations to work on single images instead of batches.

In [49]:

def make_input_batch(img, bs, h, w, ps):
    (_, _, c) = img.shape
    bat = np.zeros(bs * h * w * c).astype(np.float32).reshape((bs, h, w, c))
    for i in range(bs):
        patch_fill(bat[i], img, 0, 0, 0, ps)
        bat, crop = patch_fill(bat[i], img, 0, 0, 0, ps)
    return crop

def patch_fill(bg, img, offset_x, offset_y, gap, s):
    if bg.shape[0] != bg.shape[1]:
        exit('Error filling patches, background or image must be square')
    crop = []
    jump = s + gap
    cnt = int(bg.shape[0] / (jump))
    for j in range(cnt):
        for i in range(cnt):
            crop_temp = random_crop(img, s, s)
            crop.append(crop_temp)
            merge(bg, random_crop(img, s, s),
                i * jump + offset_x, j * jump + offset_y)
           
    return bg, crop

def random_crop(img, xl, yl):
    y = int((img.shape[0] - yl) * random.random())
    x = int((img.shape[1] - xl) * random.random())
    return crop(img, x, y, xl, yl)

def merge(bg, img, x, y):
    bg[y:y+img.shape[0] if y+img.shape[0] < bg.shape[0] else bg.shape[0],
        x:x+img.shape[1] if x+img.shape[1] < bg.shape[1] else bg.shape[1],:] =\
            img[:img.shape[0] if y+img.shape[0] < bg.shape[0] else bg.shape[0] - y,
        :img.shape[1] if x+img.shape[1] < bg.shape[1] else bg.shape[1] - x,:]

def crop(img, x, y, xl, yl):
    if (x + xl) > img.shape[1] or (y + yl) > img.shape[0]:
        return None
    return img[y:y+yl, x:x+xl,:]

## Functions for calculating the S-Score. 
The S-Score wasn't described very well in the paper nor did they publish the code. We developed this ourself

In [67]:
def sscore(input_patches, output_patches):
    #Local binary pattern settings
    radius = 1
    n_points = 8 * radius
    
    #init lists
    lbp_input = []
    lbp_output = []
    minimum_scores = []
    minimum_scores_chi = []
    
    #For every input and output batch calculate the local binary pattern.
    for i,j in zip(input_patches, output_patches):
        lbp_input.append(lbp_histogram(i, radius, n_points))
        lbp_output.append(lbp_histogram(j, radius, n_points))
     
    #For every output patch find the most similar input patch (based on distance)
    for i in lbp_output:
        score_distance = []
        score_distance_chi = []
        for j in lbp_input:
            score_distance.append(distance.euclidean(i,j))
            i = np.float32(i)
            j = np.float32(j)
            
        
            score_distance_chi.append(cv2.compareHist(i,j, 1))
            
        minimum_scores.append(min(score_distance))
        minimum_scores_chi.append(min(score_distance_chi))
        
    #Calculate the average score over all output patches        
    average_score_euclidean = sum(minimum_scores)/len(minimum_scores)
    average_score_chi = sum(minimum_scores_chi)/len(minimum_scores_chi)
    return average_score_euclidean, average_score_chi


def lbp_histogram(color_image, radius, n_points):
    #Extract the gray scale of the patch
    img = color.rgb2gray(color_image)
    #Create the local binary pattern based on the gray scale
    patterns = local_binary_pattern(img, n_points, radius)
    #Create the histogram with the same amount of bins as the pixel intensity range
    hist, _ = np.histogram(patterns, bins=256, density=True)
    return hist


def one_style_batch_score(input_image_path, path_output, patch_size, batch_size):
    
    input_image = cv2.imread(input_image_path)
    #normalize the image
    input_image = (input_image /255.0)
    crop_input = make_input_batch(input_image, 1, 256, 256, patch_size)
    
    score_list = [] 
    score_chi = []
    counter = 0 
    #Loop over the style transfered batch and calculate the s-score compared to the input style image.
    filenames = random.sample(os.listdir(path_output), batch_size)
    for filename in filenames:
        clear_output(wait=True)
        counter += 1
        print("Progress", (counter/len(filenames)) *100, "%")
        
        output_image = cv2.imread(os.path.join(path_output,filename))
        
        #normalize the image 
        output_image = (output_image /255.0)
        #Create random patches
        crop_output = make_input_batch(output_image,1 ,256,256, patch_size)
        #Calculate both s-scores
        euc, chi = sscore(crop_input, crop_output)
        score_list.append(euc)
        score_chi.append(chi)
        
    average = sum(score_list)/len(score_list)
    average_chi = sum(score_chi)/len(score_chi)
            
    return average, average_chi

#Function for running finding the s-score with a fixed batch of style images
def multi_style_fixed_batch_score (path_input, path_output, patch_size, batch_size_input, batch_size_output):
    score_list = []
    score_chi = []
    filenames_output = random.sample(os.listdir(path_output), batch_size_output)
    filenames_input = random.sample(os.listdir(path_input), batch_size_input)
    counter = 0
    
    #loop over the random sampled batch of transfered images. 
    #and calculate the s-score for the random sampled style images.
    for filename_output in filenames_output:
        output_image = cv2.imread(os.path.join(path_output,filename_output))
        output_image = (output_image /255.0)
        crop_output = make_input_batch(output_image,1 ,256,256, patch_size)
        
        clear_output(wait=True)
        counter += 1
        print("Progress", (counter/len(filenames_output)) *100, "%")
        
        for filename_input in filenames_input:
            input_image = output_image = cv2.imread(os.path.join(path_input,filename_input))
            input_image = (input_image /255.0)
            crop_input = make_input_batch(input_image, 1, 256, 256, patch_size)
            euc, chi = sscore(crop_input, crop_output)
            score_list.append(euc)
            score_chi.append(chi)           
    
    average = sum(score_list)/len(score_list)
    average_chi = sum(score_chi)/len(score_chi)
    return average, average_chi

#Function for calculating the S-score using random permutation for the input style images. 
def multi_style_random_batch_score (path_input, path_output, patch_size, batch_size_input, batch_size_output):
    
    score_list = []
    score_chi = []
    filenames_output = random.sample(os.listdir(path_output), batch_size_output)
    counter = 0
    
    #loop over the random sampled batch of transfered images. 
    #and calculate the s-score for the random sampled style images.
    for filename_output in filenames_output:
        output_image = cv2.imread(os.path.join(path_output,filename_output))
        output_image = (output_image /255.0)
        crop_output = make_input_batch(output_image,1 ,256,256, patch_size)
        
        #Report progress
        clear_output(wait=True)
        counter += 1
        print("Progress", (counter/len(filenames_output)) *100, "%")
        
        
        filenames_input = random.sample(os.listdir(path_input), batch_size_input)
        for filename_input in filenames_input:
            input_image = output_image = cv2.imread(os.path.join(path_input,filename_input))
            input_image = (input_image /255.0)
            crop_input = make_input_batch(input_image, 1, 256, 256, patch_size)
            euc, chi = sscore(crop_input, crop_output)
            score_list.append(euc)
            score_chi.append(chi)
            
            
    
    average = sum(score_list)/len(score_list)
    average_chi = sum(score_chi)/len(score_chi)
    return average, average_chi
    



## One style image run



In [69]:
##One style image score run

#Paths to input and style image
path_monet = 'Monet/6e0429f92e.jpg'
path_output = Path('Output')

#Settings
patch_size = 16
batch_size = 10 #Should be the same as patch_size in network run

#Start score run
start = time.time()
average_score,chi = one_style_batch_score(path_monet, path_output, patch_size, batch_size)
end = time.time()

#Print run stats
print('average score: ', average_score)
print('chi score', chi)
print("batch size: ", batch_size)
print("patch size: ", patch_size)
print("runtime:",end - start)



Progress 100.0 %
average score:  0.07698897557418125
chi score 0.4177838387634756
batch size:  10
patch size:  16
runtime: 8.529469013214111


## Multiple style images run

For multiple style images random sampling is used for computing time reasons

In [22]:
#Paths to in put and output
path_monet = Path('Monet')
path_output = Path('Output')

#Settings
patch_size = 16
batch_size_input = 2    #Random sample size
batch_size_output = 10

#Start score run
start = time.time()
average_score = multi_style_fixed_batch_score(path_monet, path_output, patch_size, batch_size_input, batch_size_output)
end = time.time()

#Print run stats
print('average score: ', average_score)
print("input batch size: ", batch_size_input)
print("output batch size: ", batch_size_output)
print("patch size: ", patch_size)
print("runtime:",end - start)


Progress 100.0 %
average score:  0.07578027821304481
input batch size:  2
output batch size:  10
patch size:  16
runtime: 14.076018333435059


## Multi style image run with random permutation of input image (This version is used in our report)

Settings chosen for our experiments:

patch size: 16 

sampling size for style images: 10

sampling size: 300

In [72]:
#Paths to in put and output
path_monet = Path('Monet')
path_output = Path('Output')

#Settings
patch_size = 16
#Change this value to 1 for a single style image input
batch_size_input = 10
batch_size_output = 300

#Start score run
start = time.time()
average_score, chi = multi_style_fixed_batch_score(path_monet, path_output, patch_size, batch_size_input, batch_size_output)
end = time.time()

#Print run stats
print('average score: ', average_score)
print('average chi:', chi)
print("input batch size: ", batch_size_input)
print("output batch size: ", batch_size_output)
print("patch size: ", patch_size)
print("runtime:",end - start)

Progress 100.0 %
average score:  0.07564586730144764
average chi: 0.4132278325084745
input batch size:  10
output batch size:  5
patch size:  16
runtime: 39.78747487068176
